# Homework 3 (100 points)

The goal of this homework is to practice techniques relating to SVD.

## Exercise 1 (65 points)

a) Fetch the "mnist_784" data and store is as a `.csv` (that way you don't have to fetch it every time - which takes about 30s). (4 points)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from os.path import exists
from sklearn.cluster import KMeans

In [ ]:
if not exists('mnist.csv'):
    X, y = fetch_openml(name="mnist_784", version=1,
                        return_X_y=True, as_frame=False)
    pd.DataFrame(X).join(pd.DataFrame({'Label': y})).to_csv('mnist.csv', index=False)

df = pd.read_csv('mnist.csv')

In [ ]:
df.head()

b) Plot the singular value plot for a single example of the 0 digit (5 points)

In [ ]:
zero = df[df['Label'] == 0].sample()
zero = zero.iloc[:, :784].values.reshape(28, 28)

u, s, v = np.linalg.svd(zero)
plt.plot(s)
plt.xlabel('ith Singular Value')
plt.ylabel('Singular Values')

c) By setting some singular values to 0, plot the approximation of the 0 digit next to the original digit. (5 points)

In [ ]:
rank = 6
compressed_zero = u[:, :rank] @ np.diag(s[:rank]) @ v[:rank]

fig, axes = plt.subplots(1, 2)
axes[0].imshow(zero)
axes[0].set_axis_off()
axes[1].imshow(compressed_zero)
axes[1].set_axis_off()
fig.suptitle('Original vs Compressed')

d) Consider the entire dataset as a matrix. Perform SVD and store the dataset approximation in a new `.csv` file. Explain why / how you chose a particular rank. (10 points)

In [ ]:
def compressImage(image, rank=28):
    u, s, v = np.linalg.svd(image.reshape(28, 28))
    newImage = u[:, :(rank + 1)] @ np.diag(s[:(rank + 1)]) @ v[:(rank + 1)]
    return newImage.reshape(784)

def compressData(data, rank=28):
    compressedData = []
    for row in data:
        compressedData.append(compressImage(row, rank))
    return np.array(compressedData)

In [ ]:
images = []
for i in range(10):
    images.append(df[df['Label'] == i].sample())

fig, axes = plt.subplots(1, 10)
for ax, digit, img in zip(axes, range(10), images):
    ax.set_xlabel(xlabel=str(digit))
    _, s, _ = np.linalg.svd(img.iloc[:, :784].values.reshape(28, 28))
    ax.plot(s)

fig.set_figwidth(25)

In [ ]:
if not exists('compressed_mnist.csv'):
    pd.DataFrame(compressData(df.iloc[:, :784].values, 9)).join(df['Label']).to_csv('compressed_mnist.csv', index=False)

df_compressed = pd.read_csv('compressed_mnist.csv')

In [ ]:
df_compressed.head()

One would probably look at the singular values on a graph, for each digit, and pick the number of singular values to keep, in order to have a relatively good reconstruction of each digit thus not losing too much information.
I chose to compress to rank 9.

e) As in homework 2, using Kmeans on this new dataset, cluster the images from d) using 10 clusters and plot the centroid of each cluster. (10 points)

In [ ]:
kmeans_compressed = KMeans(n_clusters=10).fit(df_compressed.iloc[:, :784].values)

fig, axes = plt.subplots(1, 10)
for ax, digit in zip(axes, kmeans_compressed.cluster_centers_):
  ax.set_axis_off()
  img = digit.reshape(28, 28)
  ax.imshow(img)

fig.set_figwidth(15)
fig.set_figheight(15)

f) Repeat e) on the original dataset. Comment on any differences (or lack thereof) you observe. (8 points)

In [ ]:
kmeans = KMeans(n_clusters=10).fit(df.iloc[:, :784].values)

fig, axes = plt.subplots(1, 10)
for ax, digit in zip(axes, kmeans.cluster_centers_):
  ax.set_axis_off()
  img = digit.reshape(28, 28)
  ax.imshow(img)

fig.set_figwidth(15)
fig.set_figheight(15)

The kmeans seems to be performing similarly across the compressed and the original datasets

g) Compare the disagreement distance of the clustering obtained in e) to the true labels, to the disagreement distance of the clustering obtained in f) to the true labels. Comment briefly. (8 points)

In [ ]:
def disagreement_dist(P_labels, C_labels):
    """
    Calculate the disagreement distance between `P_Labels` and `C_Labels`
    """
    ans = 0
    num_of_classes = len(set(P_labels))
    df_temp = pd.DataFrame({'Label': P_labels}).join(
        pd.DataFrame({'Predicted Label': C_labels}))
    
    for i in range(num_of_classes):
        temp = df_temp[df_temp['Label'] == i]
        s1 = temp['Predicted Label'].value_counts()
        nums = 0
        sum = s1.sum()
        for val in s1.values:
            nums += val
            ans += val * (sum - nums)

        temp = df_temp[df_temp['Predicted Label'] == i]
        s1 = temp['Label'].value_counts()
        nums = 0
        sum = s1.sum()
        for val in s1.values:
            nums += val
            ans += val * (sum - nums)
        
    return ans

In [ ]:
compressed_dist = disagreement_dist(df_compressed['Label'].values, kmeans_compressed.labels_)
original_dist = disagreement_dist(df['Label'].values, kmeans.labels_)

print('The disagreement distance of the compressed image clustering is: {}'.format(compressed_dist))
print('The disagreement distance of the original image clustering is: {}'.format(original_dist))

The distances seem to be similar. After running it a few times with different clusterings, they produce similar results. (They switch on being the better one)

h) Create a matrix that is the difference between the original dataset and the rank-10 approximation of the dataset. (10 points)

In [ ]:
df_compressed = pd.DataFrame(compressData(df.iloc[:, :784].values, rank=10)).join(
    df["Label"]
)

# ???

diff_matrix = df.iloc[:, :784].values - df_compressed.iloc[:, :784].values

i) The largest (using euclidean distance from the origin) rows of the matrix could be considered anomalous data points. Briefly explain why. Plot the 10 images responsible for the 10 largest rows. (5 points)

In [ ]:
def calcDistance(diff):
    distances = []
    for row in diff:
        distances.append(np.linalg.norm(row))
    return pd.Series(distances)

df['Distance'] = calcDistance(diff_matrix)
df.head()

In [ ]:
df = df.sort_values(by=['Distance'], ascending=False)

fig, axes = plt.subplots(1, 10)
for ax, digit in zip(axes, df.iloc[:, :784].values):
  ax.set_axis_off()
  img = digit.reshape(28, 28)
  ax.imshow(img)

fig.set_figwidth(15)
fig.set_figheight(15)

These images seem to be anomalous because they are very deformed compared to the other images. They are either too thin or too thick and it is difficult to understand which digit is which.

## Exercise 2 (35 points)

a) Modify the code below to pick 4 categories of news articles that you think are minimally related (for example `sci.space` and `rec.sport.baseball`). (3 points)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd


categories = ['comp.graphics', 'rec.autos', 'sci.space', 'talk.politics.guns']
news_data = fetch_20newsgroups(subset='train', categories=categories)


[nltk_data] Downloading package punkt to /Users/ivan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ivan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


b) Using the `SnowballStemmer`, stem the words in every article (10 points)

In [11]:
stemmer = SnowballStemmer('english', ignore_stopwords=True)
stemmed_articles = [' '.join(stemmer.stem(word) for sent in sent_tokenize(article) for word in word_tokenize(sent)) for article in news_data['data']]

In [12]:
print(len(stemmed_articles))

2317


c) Use the `TfidfVectorizer` on the stemmed articles. Set `min_df` and `max_df` to reasonable numbers and briefly explain your reasoning. Store the resulting dataset into a `.csv` file. (7 points)

In [13]:
vectoriser = TfidfVectorizer(stop_words='english', min_df=1, max_df=1.0)

vectorised_data = vectoriser.fit_transform(stemmed_articles)
centred_data = vectorised_data - np.mean(vectorised_data, axis=0)

In [14]:
pd.DataFrame(centred_data).join(pd.DataFrame({'Target' : news_data['target']})).to_csv('vectorised_data.csv', index=False)

d) For rank k ranging from 1 to 25:

1. Reduce the dimensionality of the tfidf vectorized data using a dimension reduction technique discussed in class.
2. Apply Kmeans on the reduced dataset to create 4 clusters
3. Record the disagreement distance between the clustering in 2 and the article category

Then plot the recorded disagreement distance per rank. Comment briefly. (15 points)

In [ ]:
u, s, v = np.linalg.svd(centred_data)

In [ ]:
import sys

disagreement_distance = []
for k in range(1,25):
    dim_reduced_dataset = u[:, :k] @ np.diag(s[:k]) @ v[:k]
    kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=10, random_state=0)
    kmeans.fit_predict(dim_reduced_dataset)
    labelsk = kmeans.labels_
    disagreement_distance.append(disagreement_dist(labelsk, news_data.target))
    sys.stdout.write('\r {}'.format(k))

plt.plot(range(1,25), disagreement_distance)
plt.ylabel('Disagreement')
plt.xlabel('Dimension')
plt.show()

-> answer